In [1]:
# Ensures that there is enough memory allocation for the model to load
import os
os.environ["PYTORCH_CUDA_ALLOC_CONF"] = "expandable_segments:True"

In [2]:
from transformers import LlavaNextProcessor, LlavaNextForConditionalGeneration
import torch
from PIL import Image
import requests

processor = LlavaNextProcessor.from_pretrained("llava-hf/llava-v1.6-mistral-7b-hf")

model = LlavaNextForConditionalGeneration.from_pretrained("llava-hf/llava-v1.6-mistral-7b-hf", torch_dtype=torch.float16, low_cpu_mem_usage=True) 
model.to("cuda:0")

/home/jovyan/myenv/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Using a slow image processor as `use_fast` is unset and a slow processor was saved with this model. `use_fast=True` will be the default behavior in v4.48, even if the model was saved with a slow processor. This will result in minor differences in outputs. You'll still be able to use a slow processor with `use_fast=False`.
Loading checkpoint shards: 100%|██████████| 4/4 [00:03<00:00,  1.26it/s]


LlavaNextForConditionalGeneration(
  (vision_tower): CLIPVisionModel(
    (vision_model): CLIPVisionTransformer(
      (embeddings): CLIPVisionEmbeddings(
        (patch_embedding): Conv2d(3, 1024, kernel_size=(14, 14), stride=(14, 14), bias=False)
        (position_embedding): Embedding(577, 1024)
      )
      (pre_layrnorm): LayerNorm((1024,), eps=1e-05, elementwise_affine=True)
      (encoder): CLIPEncoder(
        (layers): ModuleList(
          (0-23): 24 x CLIPEncoderLayer(
            (self_attn): CLIPSdpaAttention(
              (k_proj): Linear(in_features=1024, out_features=1024, bias=True)
              (v_proj): Linear(in_features=1024, out_features=1024, bias=True)
              (q_proj): Linear(in_features=1024, out_features=1024, bias=True)
              (out_proj): Linear(in_features=1024, out_features=1024, bias=True)
            )
            (layer_norm1): LayerNorm((1024,), eps=1e-05, elementwise_affine=True)
            (mlp): CLIPMLP(
              (activation_fn

In [3]:
import json
def save_labels(labels_list, file,):
    # Saves the labels in a json format 
    data ={
        'Labels' : labels_list
    }
    
    new_path = '/home/jovyan/Evaluation/Data'
    
    if not os.path.exists(new_path):
        os.makedirs(new_path) # Create the directory if it doesn't exist
        print(f"{new_path} successfully created")
    
    json_filename = file
    json_filename = f'{json_filename}.json'
    # Creates the new json file.
    file_name = os.path.join(new_path , json_filename)
    with open(file_name , 'w') as json_file:
            json.dump(data, json_file, indent=4)
    print(f"File : {json_filename} successfully created!")

In [7]:
import re
# Path to the directory (library)
directory_path = "/home/jovyan/images"
# List all files in the directory
files = os.listdir(directory_path)

# Filter out directories and show only files
files = [file for file in files if os.path.isfile(os.path.join(directory_path, file))]

# Print the list of files
for index, file in enumerate(files):
    # Optimize memory usage 
    torch.cuda.empty_cache()
    print(f"Processing image {file}")
    
    image_path = os.path.join(directory_path, file)
    image = Image.open(image_path)

    # Prompt for the Llava-hf
    prompt = """Analyze the given image and identify 10 objects in the image.
    Provide a unique , non-repetitive list of detected objects,
    ensuring each object is named using commonly understood terms.
    Avoid unnecessary descriptions—only list object names. 
    Example objects include 'chair,' 'table,' 'car,' 'tree,' and 'cup.'
    If an object is unclear, do not list it. 
    Do not list duplicate objects, even if multiple instances exist."""
    
    prompt = f"""Detect 10 the objects in the image, ensure the answers are short and concise.
    Do not include fullstops."""
    
    conversation = [
        {

          "role": "user",
          "content": [
              {"type": "text", "text": "Detect 10 simple and generic objects in the image, ensure the answers are short and concise. Do not include fullstops.Do not repeat the listed objects. If there are no other objects found, do not list anymore."},
              {"type": "image"},
            ],
        },
    ]
    prompt = processor.apply_chat_template(conversation, add_generation_prompt=True)

    inputs = processor(images=image, text=prompt, return_tensors="pt").to("cuda:0")

    # autoregressively complete prompt
    output = model.generate(**inputs, max_new_tokens=500)

    answer = processor.decode(output[0], skip_special_tokens=True)
    # Attains only the output and don't include the prompt 
    skip = len(prompt) - 6 
    answer = answer[skip:]
    
    # Converts all the text into lower caps.
    answer = answer.lower()
    # Removes the numbering 
    cleaned_text = re.sub(r'^\d+\.\s*', '', answer, flags=re.MULTILINE)
    labels_list = cleaned_text.strip('1. ').split('\n')
    # To save the labels in a list 
    print(labels_list)
    save_labels(labels_list,file)
    
    print(f"Successfully processed Image {file} , {index+1}/{len(files)} Images Processed")
     
    
print(f"All {len(files)} have been processed.")

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


Processing image 588.JPG


Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


['dog', 'table', 'mug', 'fire', 'chair', 'microphone', 'speaker', 'desk', 'hat', 'fire extinguisher']
File : 588.JPG.json successfully created!
Successfully processed Image 588.JPG , 1/265 Images Processed
Processing image 708.png


Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


...

['microphone', 'suit', 'tie', 'watch', 'ring', 'glasses', 'shirt', 'vest', 'jacket', 'seat']
File : 552.jpg.json successfully created!
Successfully processed Image 552.jpg , 262/265 Images Processed
Processing image 658.jpg


Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


['man', 'jacket', 'hand', 'mouth', 'beard', 'ear', 'nose', 'eye', 'hair', 'collar']
File : 658.jpg.json successfully created!
Successfully processed Image 658.jpg , 263/265 Images Processed
Processing image 719.png
['man', 'beard', 'glasses', 'shirt', 'jacket', 'hair', 'mustache', 'blurred background', 'lighting', 'text']
File : 719.png.json successfully created!
Successfully processed Image 719.png , 264/265 Images Processed
Processing image 579.jpg


Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


['trees', 'sky', 'leaves', 'clouds', 'grass', 'trunk', 'jacket', 'hair', 'beard', 'shirt']
File : 579.jpg.json successfully created!
Successfully processed Image 579.jpg , 265/265 Images Processed
All 265 have been processed.
